In [1]:
%run utils/Plot_utils.ipynb 

SelectMultiple(description='Combinations:', layout=Layout(height='350px', width='90%'), options=('Model: gpt-3…

Button(description='Load Data and Plot', style=ButtonStyle())

Output()

Button(description='Show Table', style=ButtonStyle())

Output()